<a href="https://colab.research.google.com/github/flrain2/Study/blob/main/movie_recommend(funkSVD).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

truncatedSVD: https://lsjsj92.tistory.com/570

svd추천(한글): https://data-science-hi.tistory.com/82

https://chaelist.github.io/docs/ml_application/topic_modeling/

naver review: https://chaelist.github.io/docs/ml_application/sentiment_analysis/

https://diane-space.tistory.com/90

https://www.kaggle.com/alsojmc/movie-recommender-systems

from korea_news_crawler.articlecrawler import ArticleCrawler: https://github.com/lumyjuwon/KoreaNewsCrawler

naver: https://kyounghwan01.github.io/blog/etc/python/naver-news-crawling/#%E1%84%8F%E1%85%A9%E1%84%83%E1%85%B3

naver review crawling: https://ysyblog.tistory.com/59

naver review: https://chaelist.github.io/docs/ml_application/sentiment_analysis/

# naver movie score 크롤링

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import random
import requests
from bs4 import BeautifulSoup
import time
import csv

In [ ]:
base_url = 'https://movie.naver.com/movie/point/af/list.nhn?&page={}'

file_list =[]
for page in range(0,100):
  url = base_url.format(page)
  res=requests.get(url)
  if res.status_code == 200:
    soup = BeautifulSoup(res.text,'lxml')
    tds = soup.select('table.list_netizen > tbody > tr ')

    for td in tds:
      id = td.select_one('td.num > a.author').text.strip()
      movie_title = td.select_one('td.title > a.movie').text.strip()
      score = td.select_one('td.title > div.list_netizen_score > em').text.strip()
      comment = td.select_one('td.title > br').next_sibling.strip()

      # data append to list
      data = [id, movie_title, score, comment]
      file_list .append(data)

    interval = round(random.uniform(0.2,1.2),2)
    time.sleep(interval)

In [ ]:
# to csv
with open('/content/drive/MyDrive/Colab Notebooks/review.csv', 'w', encoding='utf-8', newline='') as csvfile:
    writer = csv.writer(csvfile)
    #write header
    writer.writerow(['id','movie_title', 'score', 'comment'])
    for i in file_list:
        writer.writerow(i)

# 1. data information

In [ ]:
import pandas as pd
import numpy as np
from math import sqrt

In [ ]:
movie = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/review.csv')

In [ ]:
movie

,id,movie_title,score,comment
0,wlgu****,소년시절의 너,10,여운이 정말 긴 영화..
1,wehl****,뚜르: 내 생애 최고의 49일,10,NaN
2,yong****,빅 피쉬,9,우리가 팀버튼의 환상을 사랑하는 이유
3,jaki****,도둑들,10,한국에 이정도로 완성도 높고 스케일 큰 블록버스터 액션 영화가 다시 나올거 같지 않...
4,bard****,크루엘라,10,기대일도없었는데 재미있었어요
...,...,...,...,...
995,hikm****,낙원의 밤,10,역시 박훈정 감독님!!
996,gogy****,천공의 성 라퓨타,10,정말 아름다웠다. OST도 찾아들었다.
997,pkot****,분노의 질주: 더 얼티메이트,6,억지 신파히어로물보는줄 알았다뇌를 비우고 보면 킬링타임은 될듯
998,doro****,그래비티,10,굿 글자수제한없애주세오


In [ ]:
unique_len = movie.apply(lambda x : len(x.unique()))
unique_len

id             882
movie_title    373
score           10
comment        911
dtype: int64

In [ ]:
movie.isnull().sum()

id              0
movie_title     0
score           0
comment        78
dtype: int64

In [ ]:
movie['movie_title'].unique()

array(['소년시절의 너', '뚜르: 내 생애 최고의 49일', '빅 피쉬', '도둑들', '크루엘라', '아일라',
       '새콤달콤', '분노의 질주: 더 얼티메이트', '눈부신 세상 끝에서, 너와 나', '루시', '반딧불이의 묘',
       '버닝', '컨저링 3: 악마가 시켰다', '자산어보', '레퀴엠', '곤지암', '라야와 마지막 드래곤',
       '사랑의 선물', '피아니스트', '트루먼 쇼', '메이헴', '더 스파이', '원더 휠',
       '내니 맥피 2 - 유모와 마법소동', '찬실이는 복도 많지', '1987', '전야', '레이디 버드',
       '붉은 돼지', '마루 밑 아리에티', '아들의 이름으로', '연평해전', '구직자들', '범털2 : 쩐의 전쟁',
       '천문: 하늘에 묻는다', '인트로덕션', '낫아웃', '보이저스', '2021 스웨그 에이지 : 외쳐, 조선!',
       '#위왓치유', '실버라이닝 플레이북', '어른들은 몰라요', '파이프라인', '행복', '101 달마시안',
       '더캐스팅', '일주일간 친구', '너는 여기에 없었다', '아이, 토냐', '바람', '공조', '말모이',
       '보디 히트', '폭력의 역사', '메트로: 마지막 탈출', '캡틴 마블', '극장판 귀멸의 칼날: 무한열차편',
       '불한당: 나쁜 놈들의 세상', '비포 선라이즈', 'TV소설 큰 언니', '낙원의 밤', '밀월', '컨저링',
       '괴물', '뱅가드', '극장판 콩순이: 장난감나라 대모험', '아마겟돈', '써커 펀치',
       '해리 포터와 마법사의 돌', '8년을 뛰어넘은 신부', '국가대표', '용서는 없다', '글래디에이터',
       '아미 오브 더 데드', '서편제', '프로페서 앤 매드맨', '디파티드', '아가씨', '분노의 질주: 홉스&쇼',
       '먼 훗날 우리', '라이더스 오브 저스티스', '클레멘타인', '

#funkSVD

In [ ]:
def FunkSVD(ratings_mat, latent_features=50, learning_rate=0.0001, iters=100, random_seed=52, verbose=True):
    '''This function performs matrix factorization using a basic form of FunkSVD with no regularization
    
    INPUT:
    ratings_mat - (numpy array) a matrix with users as rows, offers (subtype) as columns, and ratings as values
    latent_features - (int) the number of latent features used
    learning_rate - (float) the learning rate 
    iters - (int) the number of iterations
    
    OUTPUT:
    user_mat - (numpy array) a user by latent feature matrix
    offer_mat - (numpy array) a latent feature by offer matrix
    '''
    
    # Set up useful values to be used through the rest of the function
    n_users = ratings_mat.shape[0]
    n_offers = ratings_mat.shape[1]
    num_ratings = np.count_nonzero(~np.isnan(ratings_mat))
    
    # initialize the user and offer matrices with random values
    np.random.seed (random_seed)
    user_mat = np.random.rand(n_users, latent_features)
    offer_mat = np.random.rand(latent_features, n_offers)
    
    # initialize sse at 0 for first iteration
    sse_accum = 0
    
    # header for running results
    if verbose:
        print("Optimizaiton Statistics")
        print("Iterations | Mean Squared Error ")
    
    # for each iteration
    for iteration in range(iters):

        # update our sse
        old_sse = sse_accum
        sse_accum = 0
        
        # For each user-offer pair
        for i in range(n_users):
            for j in range(n_offers):
                
                # if the rating exists
                if (ratings_mat[i, j] == 0)|(ratings_mat[i, j] > 0):
                    
                    # compute the error as the actual minus the dot product of the user and offer latent features
                    diff = ratings_mat[i, j] - np.dot(user_mat[i, :], offer_mat[:, j])
                    
                    # Keep track of the sum of squared errors for the matrix
                    sse_accum += diff**2
                    
                    # update the values in each matrix in the direction of the gradient
                    for k in range(latent_features):
                        user_mat[i, k] += learning_rate * (2*diff*offer_mat[k, j])
                        offer_mat[k, j] += learning_rate * (2*diff*user_mat[i, k])

        # print results for iteration
        if verbose: 
            print("%d \t\t %f" % (iteration+1, sse_accum / num_ratings))
    
    return user_mat, offer_mat 

def validation_func (test_matrix, predict_train_matrix, is_train =False, verbose=True):
    """Function that runs validation of predicted user item matrix (predict_train_matrix) on the given user item matrix 
    (test or train matrix). 
    
    INPUT: 
    Test user item matrix, predicted user item matrix, is_train sign (if validation done on train), verbose sign (whether to print results)
    OUTPUT: 
    MSE error value, Accuracy value. If verbose equals True then results just printed. Nothing returned.
    Rating이면 accuracy를 차이/원값 비율로, 
    있는지의 여부이면, 맞춘 비율.       

    """
        
    #will be looping over common customers only
    test_idx = test_matrix.index.intersection (predict_train_matrix.index)
    test_cols = test_matrix.columns

    sse_accum = 0
    diff = 0
    acc = []

    for i in test_idx: 
        for j in test_cols: 
            if test_matrix.at [i,j]>=0:
                # compute the error as the actual minus the dot product of the user and offer latent features
                diff = test_matrix.at [i,j] - predict_train_matrix.at [i,j]
                        
                # Keep track of the sum of squared errors for the matrix
                sse_accum += diff**2
                acc.append (abs (diff))

    #deviding on the overall not null values in test_matrix
    mse_error = sse_accum/(test_matrix.notnull().sum ().sum())
    acc_test = 1 - sum (acc)/(test_matrix.notnull().sum ().sum())

    if (is_train == False)&(verbose): 
        print ('\n')
        print ('MSE on the test set is: ', mse_error)
        print ('RMSE on the test set is: ', sqrt(mse_error))
        print ('Accuracy(0/1) on the test set is: ', acc_test)
        print ('\n')
    if (is_train == True)&(verbose): 
        print ('\n')
        print ('MSE on the train set is: ', mse_error)
        print ('RMSE on the train set is: ', sqrt(mse_error))
        print ('Accuracy(0/1) on the train set is: ', acc_test)
        print ('\n')
    #if no need for printing just return values of MSE and Accuracy
    if verbose == False: 
        return mse_error, acc_test

In [ ]:
# [id & movie_title] 숫자로 변형
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()

reviews_with_comment_df = movie
reviews_with_comment_df['id'] = label_encoder.fit_transform(movie['id'])
reviews_with_comment_df['movie_num'] = label_encoder.fit_transform(movie['movie_title'])

# comment 제거
movie_user = reviews_with_comment_df[['id','movie_num','score']]

In [ ]:
movie_user.drop_duplicates(inplace=True) #(981756, 3) 에서 (980112, 3) 줄어듬
print('we have',movie_user.shape[0], 'ratings')
print('the number of unique users we have is:', len(movie_user.id.unique()))
print('the number of unique movie we have is:', len(movie_user.movie_num.unique()))
print("The median user rated %d movie."%movie_user.id.value_counts().median())
print('The max rating is: %d'%movie_user.score.max(),"the min rating is: %d"%movie_user.score.min())
movie_user.head()

we have 985 ratings
the number of unique users we have is: 882
the number of unique movie we have is: 373
The median user rated 1 movie.
The max rating is: 10 the min rating is: 1


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,id,movie_num,score
0,807,212,10
1,787,111,10
2,855,192,9
3,338,102,10
4,67,327,10


In [ ]:
movie_user=movie_user.groupby(['id','movie_num'])['score'].sum().unstack()

In [ ]:
#Number of zeroes and ones in the matrix  영화별로 1,2,3,4,5 를 얼마나 % 받았는지
#for col in user_item_matrix.columns:
for col in movie_user.columns:
    print (movie_user [col].value_counts()/movie_user [col].notnull ().sum()*100)

10.0    66.666667
8.0     33.333333
Name: 0, dtype: float64
8.0    100.0
Name: 1, dtype: float64
9.0    50.0
7.0    50.0
Name: 2, dtype: float64
10.0    100.0
Name: 3, dtype: float64
1.0     50.0
10.0    50.0
Name: 4, dtype: float64
7.0    100.0
Name: 5, dtype: float64
1.0     66.666667
10.0    33.333333
Name: 6, dtype: float64
3.0    100.0
Name: 7, dtype: float64
10.0    100.0
Name: 8, dtype: float64
6.0    100.0
Name: 9, dtype: float64
10.0    100.0
Name: 10, dtype: float64
6.0    100.0
Name: 11, dtype: float64
2.0    100.0
Name: 12, dtype: float64
10.0    100.0
Name: 13, dtype: float64
2.0    100.0
Name: 14, dtype: float64
9.0    100.0
Name: 15, dtype: float64
10.0    100.0
Name: 16, dtype: float64
10.0    100.0
Name: 17, dtype: float64
10.0    66.666667
2.0     33.333333
Name: 18, dtype: float64
10.0    100.0
Name: 19, dtype: float64
9.0    100.0
Name: 20, dtype: float64
4.0    50.0
1.0    50.0
Name: 21, dtype: float64
2.0    100.0
Name: 22, dtype: float64
10.0    100.0
Name: 23, d

In [ ]:
#이제 FUNK SVD를 적용함
user_mat, offer_mat =FunkSVD(np.matrix(movie_user), latent_features=8, learning_rate=0.01, iters=10)

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 28.360153
2 		 16.748700
3 		 10.893358
4 		 6.979964
5 		 4.290517
6 		 2.526702
7 		 1.436444
8 		 0.802508
9 		 0.452345
10 		 0.263850


In [ ]:
np.round (np.dot(user_mat, offer_mat))

array([[ 6.,  5.,  7., ...,  8.,  7.,  5.],
       [ 9.,  7.,  8., ..., 10.,  9.,  6.],
       [ 9.,  6.,  9., ..., 11.,  8.,  6.],
       ...,
       [ 8.,  6.,  8., ..., 10.,  8.,  6.],
       [ 2.,  1.,  1., ...,  1.,  1.,  1.],
       [ 6.,  4.,  6., ...,  7.,  5.,  4.]])

In [ ]:
#이제 예측치를 만듬
train_predict_np= np.abs(np.round (np.dot(user_mat, offer_mat)))

In [ ]:
train_predict_np

array([[ 6.,  5.,  7., ...,  8.,  7.,  5.],
       [ 9.,  7.,  8., ..., 10.,  9.,  6.],
       [ 9.,  6.,  9., ..., 11.,  8.,  6.],
       ...,
       [ 8.,  6.,  8., ..., 10.,  8.,  6.],
       [ 2.,  1.,  1., ...,  1.,  1.,  1.],
       [ 6.,  4.,  6., ...,  7.,  5.,  4.]])

In [ ]:
#checking predicted values in each column, it should be filled only with zeroes and ones
predict_train_matrix = pd.DataFrame (train_predict_np)

#for j in predict_train_matrix.columns:
#    print (predict_train_matrix[j].value_counts())

In [ ]:
#validation results on train and test sets    panda 로만 해야 함 
train_predict_np=pd.DataFrame(train_predict_np)

validation_func (movie_user, predict_train_matrix, is_train=True)



MSE on the train set is:  0.16276703967446593
RMSE on the train set is:  0.40344397340208954
Accuracy(0/1) on the train set is:  0.861648016276704




In [ ]:
train_predict_np

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372
0,6.0,5.0,7.0,7.0,6.0,5.0,6.0,3.0,7.0,4.0,7.0,5.0,2.0,6.0,3.0,5.0,6.0,9.0,7.0,7.0,7.0,4.0,3.0,7.0,5.0,7.0,6.0,6.0,6.0,6.0,4.0,7.0,3.0,6.0,7.0,8.0,6.0,5.0,7.0,6.0,...,7.0,6.0,7.0,5.0,7.0,7.0,8.0,6.0,7.0,5.0,6.0,6.0,5.0,6.0,8.0,7.0,5.0,6.0,6.0,3.0,6.0,2.0,5.0,6.0,6.0,3.0,6.0,7.0,3.0,5.0,7.0,8.0,7.0,6.0,6.0,7.0,8.0,8.0,7.0,5.0
1,9.0,7.0,8.0,10.0,8.0,7.0,7.0,4.0,9.0,5.0,8.0,6.0,3.0,7.0,4.0,7.0,8.0,12.0,8.0,8.0,8.0,4.0,3.0,8.0,8.0,9.0,8.0,7.0,8.0,8.0,6.0,8.0,4.0,9.0,7.0,9.0,8.0,6.0,9.0,8.0,...,10.0,7.0,9.0,7.0,9.0,9.0,10.0,9.0,7.0,7.0,8.0,8.0,6.0,9.0,10.0,8.0,6.0,8.0,9.0,4.0,7.0,3.0,6.0,6.0,8.0,5.0,9.0,9.0,4.0,6.0,9.0,10.0,8.0,8.0,9.0,9.0,10.0,10.0,9.0,6.0
2,9.0,6.0,9.0,10.0,8.0,6.0,8.0,5.0,10.0,5.0,9.0,6.0,3.0,9.0,5.0,8.0,8.0,12.0,9.0,9.0,8.0,5.0,3.0,8.0,8.0,9.0,8.0,8.0,8.0,9.0,6.0,9.0,4.0,9.0,8.0,9.0,7.0,6.0,9.0,9.0,...,10.0,7.0,9.0,8.0,10.0,9.0,11.0,9.0,8.0,6.0,8.0,9.0,7.0,8.0,10.0,7.0,7.0,8.0,9.0,5.0,8.0,2.0,6.0,7.0,8.0,4.0,9.0,10.0,4.0,7.0,9.0,10.0,9.0,8.0,9.0,8.0,10.0,11.0,8.0,6.0
3,6.0,5.0,7.0,7.0,6.0,4.0,5.0,3.0,7.0,3.0,6.0,4.0,2.0,6.0,3.0,5.0,5.0,9.0,7.0,6.0,6.0,4.0,3.0,6.0,5.0,6.0,6.0,6.0,5.0,6.0,4.0,7.0,3.0,6.0,6.0,7.0,5.0,4.0,6.0,6.0,...,7.0,5.0,7.0,5.0,7.0,7.0,8.0,6.0,6.0,4.0,6.0,6.0,5.0,6.0,8.0,6.0,4.0,6.0,6.0,3.0,5.0,2.0,5.0,5.0,6.0,3.0,6.0,7.0,3.0,5.0,7.0,7.0,6.0,6.0,6.0,6.0,7.0,8.0,6.0,5.0
4,6.0,4.0,6.0,7.0,6.0,4.0,5.0,3.0,6.0,3.0,5.0,3.0,2.0,4.0,3.0,4.0,5.0,8.0,5.0,5.0,4.0,3.0,2.0,5.0,5.0,5.0,5.0,4.0,5.0,6.0,4.0,6.0,2.0,6.0,4.0,5.0,5.0,4.0,6.0,5.0,...,6.0,4.0,6.0,5.0,7.0,6.0,7.0,5.0,5.0,4.0,5.0,5.0,4.0,5.0,6.0,5.0,4.0,4.0,6.0,3.0,4.0,2.0,4.0,4.0,5.0,3.0,6.0,7.0,2.0,4.0,5.0,6.0,5.0,5.0,6.0,5.0,6.0,7.0,5.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
877,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0
878,6.0,5.0,6.0,7.0,5.0,4.0,5.0,3.0,7.0,4.0,5.0,5.0,3.0,6.0,3.0,5.0,5.0,9.0,7.0,7.0,6.0,4.0,2.0,7.0,5.0,6.0,6.0,6.0,6.0,5.0,4.0,7.0,3.0,6.0,6.0,7.0,6.0,4.0,6.0,6.0,...,7.0,5.0,6.0,5.0,7.0,8.0,7.0,6.0,6.0,5.0,6.0,6.0,5.0,6.0,8.0,6.0,4.0,6.0,6.0,3.0,6.0,2.0,5.0,4.0,6.0,2.0,6.0,6.0,3.0,5.0,7.0,7.0,6.0,6.0,5.0,6.0,7.0,8.0,6.0,5.0
879,8.0,6.0,8.0,9.0,6.0,6.0,7.0,4.0,9.0,5.0,8.0,6.0,3.0,8.0,4.0,7.0,7.0,11.0,9.0,8.0,8.0,5.0,3.0,8.0,6.0,8.0,7.0,7.0,7.0,7.0,5.0,8.0,4.0,8.0,8.0,9.0,7.0,6.0,8.0,8.0,...,9.0,7.0,8.0,7.0,9.0,9.0,10.0,8.0,8.0,6.0,8.0,8.0,6.0,8.0,10.0,8.0,6.0,8.0,7.0,4.0,7.0,2.0,6.0,6.0,8.0,3.0,8.0,8.0,3.0,6.0,9.0,9.0,8.0,7.0,7.0,8.0,9.0,10.0,8.0,6.0
880,2.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,1.0,2.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,...,1.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,2.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [ ]:
def rec(user, top):
    global pred
    global movies
    p = train_predict_np
    m = reviews_with_comment_df
    
    tmp = pd.DataFrame(p.loc[user,:].T)
    tmp['movie_num']= tmp.index
    tmp.columns=['predict','movie_num']
    tmp.sort_values('predict', ascending=False, inplace=True)
    tmp=pd.DataFrame(tmp.head(10))
    tmp=pd.merge(tmp, m)
    tmp = tmp.drop_duplicates(['movie_num'])

    return tmp[['predict','movie_num','movie_title']]

In [ ]:
p = train_predict_np
m = reviews_with_comment_df

tmp = pd.DataFrame(p.loc[16,:].T)
tmp

,16
0,7.0
1,6.0
2,7.0
3,8.0
4,6.0
...,...
368,7.0
369,8.0
370,9.0
371,7.0


In [ ]:
tmp['movie_num']= tmp.index
tmp.columns=['predict','movie_num']

In [ ]:
tmp

,predict,movie_num
0,7.0,0
1,6.0,1
2,7.0,2
3,8.0,3
4,6.0,4
...,...,...
368,7.0,368
369,8.0,369
370,9.0,370
371,7.0,371


In [ ]:
tmp.sort_values('predict', ascending=False, inplace=True)
tmp=pd.DataFrame(tmp.head(10))
tmp=pd.merge(tmp, m)
tmp = tmp.drop_duplicates(['movie_num'])

In [ ]:
r= rec(16,10)
r

,predict,movie_num,movie_title
0,13.0,264,연평해전
61,13.0,327,크루엘라
198,13.0,200,새콤달콤
295,11.0,302,조커
310,10.0,177,분노의 질주: 더 얼티메이트
355,10.0,60,낙원의 밤
362,10.0,62,낫아웃
374,9.0,370,회사원
376,9.0,115,라야와 마지막 드래곤
378,9.0,322,컨저링 3: 악마가 시켰다


In [ ]:
reviews_with_comment_df[reviews_with_comment_df['id'] == 16]

,id,movie_title,score,comment,movie_num
420,16,애나벨: 인형의 주인,7,나이가 들어서 그런가 이런 유령이나 악마 소재 공포영화는 이제 유치하네요. ㅋㅋㅋ ...,250
